In [2]:
import plotly.offline as py
import plotly.graph_objs as graph_objs
import pandas as pd
import geopandas as gpd
import numpy as np

mapbox_access_token = "pk.eyJ1IjoiamFja3AiLCJhIjoidGpzN0lXVSJ9.7YK6eRwUNFwd3ODZff6JvA"

In [29]:
full_data_path = 'Compressed_Mortality_1999_2016.txt'
df = pd.read_csv(full_data_path, sep = '\t+', dtype = str)
df = df.replace('Suppressed', np.nan, regex=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [41]:
df['"County Code"'] = df['"County Code"'].apply(lambda x: x.strip('"'))

In [207]:
df.tail()

,"""County""","""County Code""","""Year""","""Year Code""",Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval
56641,"""Weston County, WY""",56045,"""2012""","""2012""",NaN,7065,NaN,NaN,NaN,NaN,NaN,NaN
56642,"""Weston County, WY""",56045,"""2013""","""2013""",NaN,7163,NaN,NaN,NaN,NaN,NaN,NaN
56643,"""Weston County, WY""",56045,"""2014""","""2014""",NaN,7179,NaN,NaN,NaN,NaN,NaN,NaN
56644,"""Weston County, WY""",56045,"""2015""","""2015""",NaN,7230,NaN,NaN,NaN,NaN,NaN,NaN
56645,"""Weston County, WY""",56045,"""2016""","""2016""",NaN,7236,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
df.to_csv('age_adjusted_death_rate.csv')

In [33]:
dfg = gpd.read_file('us-counties.json')

In [38]:
dfg['id'] = dfg['id'].apply(lambda x: x.zfill(5))

In [39]:
dfg.head()

,id,name,geometry
0,01001,Autauga,"POLYGON ((-86.41178600000001 32.706342, -86.41..."
1,01003,Baldwin,"POLYGON ((-87.76459 31.298768, -87.616713 31.2..."
2,01005,Barbour,"POLYGON ((-85.354736 32.147694, -85.053504 32...."
3,01007,Bibb,"POLYGON ((-87.063542 33.248559, -87.025203 33...."
4,01009,Blount,"POLYGON ((-86.488463 34.261793, -86.455601 34...."


In [52]:
df2 = pd.read_csv('Drug_Poisoning_Mortality_by_County.csv')
df2['FIPS'] = df2['FIPS'].apply(lambda x: str(x).zfill(5))

In [53]:
df2.head()

,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
1,01001,2002,Alabama,AL,1,"Autauga County, AL",45909.0,4.1-6
2,01001,1999,Alabama,AL,1,"Autauga County, AL",42963.0,2.1-4
3,01001,2007,Alabama,AL,1,"Autauga County, AL",52405.0,6.1-8
4,01001,2010,Alabama,AL,1,"Autauga County, AL",54660.0,8.1-10


In [58]:
df_2015 = df2[(df2.Year == 2015)]

In [59]:
df_2015.head()

,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
24,01003,2015,Alabama,AL,1,"Baldwin County, AL",203709.0,20.1-22
43,01005,2015,Alabama,AL,1,"Barbour County, AL",26489.0,6.1-8
61,01007,2015,Alabama,AL,1,"Bibb County, AL",22583.0,14.1-16
69,01009,2015,Alabama,AL,1,"Blount County, AL",57673.0,16.1-18


In [105]:
years = sorted(df2['Year'].unique().tolist())
years

[1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015]

In [61]:
bins = df_2015['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'].unique()

In [111]:
for yr in years:
    df_single_year = df2[(df2.Year == yr)]
    merged = dfg.merge(df_single_year, right_on='FIPS', left_on='id' )
    for bin in bins:
        print(yr,bin)
        geo_layer = merged[(merged['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'] == bin)] # geo
        try:
            geo_layer.to_file("{0}/{1}.geojson".format(yr, bin), driver='GeoJSON')
        except:
            print('err')

1999 10.1-12
1999 20.1-22
1999 6.1-8
1999 14.1-16
1999 16.1-18
1999 8.1-10
1999 12.1-14
1999 18.1-20
1999 22.1-24
1999 26.1-28
1999 24.1-26
err
1999 >30
err
1999 0-2
1999 28.1-30
err
1999 4.1-6
1999 2.1-4
2000 10.1-12
2000 20.1-22
2000 6.1-8
2000 14.1-16
2000 16.1-18
2000 8.1-10
2000 12.1-14
2000 18.1-20
2000 22.1-24
2000 26.1-28
err
2000 24.1-26
err
2000 >30
err
2000 0-2
2000 28.1-30
2000 4.1-6
2000 2.1-4
2001 10.1-12
2001 20.1-22
2001 6.1-8
2001 14.1-16
2001 16.1-18
2001 8.1-10
2001 12.1-14
2001 18.1-20
2001 22.1-24
2001 26.1-28
err
2001 24.1-26
2001 >30
2001 0-2
2001 28.1-30
err
2001 4.1-6
2001 2.1-4
2002 10.1-12
2002 20.1-22
2002 6.1-8
2002 14.1-16
2002 16.1-18
2002 8.1-10
2002 12.1-14
2002 18.1-20
2002 22.1-24
2002 26.1-28
2002 24.1-26
2002 >30
2002 0-2
2002 28.1-30
err
2002 4.1-6
2002 2.1-4
2003 10.1-12
2003 20.1-22
2003 6.1-8
2003 14.1-16
2003 16.1-18
2003 8.1-10
2003 12.1-14
2003 18.1-20
2003 22.1-24
2003 26.1-28
2003 24.1-26
2003 >30
2003 0-2
2003 28.1-30
2003 4.1-6
2003 2.1-4

In [70]:
merged = dfg.merge(df_2015, right_on='FIPS', left_on='id' )

In [71]:
type(merged), type(dfg)

(geopandas.geodataframe.GeoDataFrame, geopandas.geodataframe.GeoDataFrame)

In [72]:
merged.head()

,id,name,geometry,FIPS,Year,State,ST,FIPS State,County,Population,"Estimated Age-adjusted Death Rate, 16 Categories (in ranges)"
0,01001,Autauga,"POLYGON ((-86.41178600000001 32.706342, -86.41...",01001,2015,Alabama,AL,1,"Autauga County, AL",55347.0,10.1-12
1,01003,Baldwin,"POLYGON ((-87.76459 31.298768, -87.616713 31.2...",01003,2015,Alabama,AL,1,"Baldwin County, AL",203709.0,20.1-22
2,01005,Barbour,"POLYGON ((-85.354736 32.147694, -85.053504 32....",01005,2015,Alabama,AL,1,"Barbour County, AL",26489.0,6.1-8
3,01007,Bibb,"POLYGON ((-87.063542 33.248559, -87.025203 33....",01007,2015,Alabama,AL,1,"Bibb County, AL",22583.0,14.1-16
4,01009,Blount,"POLYGON ((-86.488463 34.261793, -86.455601 34....",01009,2015,Alabama,AL,1,"Blount County, AL",57673.0,16.1-18


In [73]:
for bin in bins:
    geo_layer = merged[(merged['Estimated Age-adjusted Death Rate, 16 Categories (in ranges)'] == bin)] # geo
    geo_layer.to_file("{0}.geojson".format(bin), driver='GeoJSON')

In [16]:
data = graph_objs.Data([
    graph_objs.Scattermapbox(
        lat=['45.5017'],
        lon=['-73.5673'],
        mode='markers',        
    )
])

layout = dict(
    height=600,
    autosize=True,
    hovermode='closest',
    margin=dict(t=0,r=0,l=0,b=0),
    mapbox=dict(
        layers=[
            dict(
                sourcetype = 'geojson',
                source = 'https://raw.githubusercontent.com/python-visualization/folium/master/tests/us-counties.json',
                type = 'fill',
                color = 'rgba(163,22,19,0.8)'
            )
        ],
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=27.8,
            lon=-83
        ),
        pitch=0,
        zoom=5.2,
        style='light'        
    )
)    

fig = dict(data=data, layout=layout)

In [17]:
py.plot(fig, filename='county-level-choropleths-python.html')

'file:///Users/jackparmer/Documents/mapbox_counties/county-level-choropleths-python.html'

In [198]:
df_lat_lon = pd.read_csv('us_county_lat_lon.txt', sep = '\t+', dtype = str, index_col = False)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [200]:
df_lat_lon['FIPS '] = df_lat_lon['FIPS '].apply(lambda x: x.zfill(5))

In [201]:
df_lat_lon['Hover'] = df_lat_lon['County '] + '<br>' + df_lat_lon['State'] + '<br>' + df_lat_lon['FIPS ']

In [202]:
df_lat_lon.head()

,Sort,State,FIPS,County,CountySeat(s),Population(2010),LandAreakm2,LandAreami2,WaterAreakm2,WaterAreami2,TotalAreakm2,TotalAreami2,Latitude,Longitude,Hover
0,1,AL,01001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,32.536382,-86.644490,Autauga<br>AL<br>01001
1,2,AL,01003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",30.659218,-87.746067,Baldwin<br>AL<br>01003
2,3,AL,01005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,31.870670,-85.405456,Barbour<br>AL<br>01005
3,4,AL,01007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,33.015893,-87.127148,Bibb<br>AL<br>01007
4,5,AL,01009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,33.977448,-86.567246,Blount<br>AL<br>01009


In [203]:
df_lat_lon.to_csv('lat_lon_counties.csv')

In [204]:
csv = pd.read_csv('lat_lon_counties.csv')

In [205]:
csv.head()

,Unnamed: 0,Sort,State,FIPS,County,CountySeat(s),Population(2010),LandAreakm2,LandAreami2,WaterAreakm2,WaterAreami2,TotalAreakm2,TotalAreami2,Latitude,Longitude,Hover
0,0,1,AL,1001,Autauga,Prattville,"54,571","1,539.582",594.436,25.776,9.952,"1,565.358",604.388,32.536382,-86.644490,Autauga<br>AL<br>01001
1,1,2,AL,1003,Baldwin,Bay Minette,"182,265","4,117.522","1,589.784","1,133.190",437.527,"5,250.712","2,027.311",30.659218,-87.746067,Baldwin<br>AL<br>01003
2,2,3,AL,1005,Barbour,Clayton,"27,457","2,291.819",884.876,50.865,19.639,"2,342.684",904.515,31.870670,-85.405456,Barbour<br>AL<br>01005
3,3,4,AL,1007,Bibb,Centreville,"22,915","1,612.481",622.582,9.289,3.587,"1,621.770",626.169,33.015893,-87.127148,Bibb<br>AL<br>01007
4,4,5,AL,1009,Blount,Oneonta,"57,322","1,669.962",644.776,15.157,5.852,"1,685.119",650.628,33.977448,-86.567246,Blount<br>AL<br>01009


In [112]:
len(bins)

16

In [112]:
len(bins)

16

In [113]:
cs = ["#171c42","#24327a","#214ea5","#006fbe","#3f8eba","#76a9be","#aac3cd","#d2d7dd",\
      "#e6d2d2","#ddb2a4","#d08b73","#c26245","#b1392a","#911a28","#670d22","#3c0911"]

In [127]:
cm = dict(zip(bins, cs))
cm

{'0-2': '#b1392a',
 '10.1-12': '#171c42',
 '12.1-14': '#aac3cd',
 '14.1-16': '#006fbe',
 '16.1-18': '#3f8eba',
 '18.1-20': '#d2d7dd',
 '2.1-4': '#3c0911',
 '20.1-22': '#24327a',
 '22.1-24': '#e6d2d2',
 '24.1-26': '#d08b73',
 '26.1-28': '#ddb2a4',
 '28.1-30': '#911a28',
 '4.1-6': '#670d22',
 '6.1-8': '#214ea5',
 '8.1-10': '#76a9be',
 '>30': '#c26245'}

In [177]:
list(bins)

['10.1-12',
 '20.1-22',
 '6.1-8',
 '14.1-16',
 '16.1-18',
 '8.1-10',
 '12.1-14',
 '18.1-20',
 '22.1-24',
 '26.1-28',
 '24.1-26',
 '>30',
 '0-2',
 '28.1-30',
 '4.1-6',
 '2.1-4']

In [183]:
df_lat_lon.columns

Index(['Sort ', 'State', 'FIPS ', 'County ', 'CountySeat(s) ',
       'Population(2010)', 'LandAreakm2 ', 'LandAreami2 ', 'WaterAreakm2 ',
       'WaterAreami2 ', 'TotalAreakm2 ', 'TotalAreami2 ', 'Latitude ',
       'Longitude'],
      dtype='object')

In [170]:
data = [dict(
    lat = df_lat_lon['Latitude '],
    lon = df_lat_lon['Longitude'],
    text = df_lat_lon['Hover'],
    marker = dict(size=3),
    opacity = 0,
    type = 'scattermapbox',    
)]

In [171]:
layout = dict(
    mapbox = dict(
        layers = [],
        accesstoken = mapbox_access_token,
        style = 'light',   
        center=dict(
            lat=38.72490,
            lon=-95.61446,
        ),
        pitch=0,
        zoom=3.4,        
    ),
    hovermode = 'closest',
    margin = dict(r=0, l=0, t=0, b=0)
)

In [172]:
base_url = 'https://raw.githubusercontent.com/jackparmer/mapbox-counties/master/'
year = '2005'
for bin in bins:
    geo_layer = dict(
        sourcetype = 'geojson',
        source = base_url + year + '/' + bin + '.geojson',
        type = 'fill',
        color = cm[bin],
        opacity = 0.5
    )
    layout['mapbox']['layers'].append(geo_layer)

In [173]:
fig = dict(data=data, layout=layout)
py.plot(fig, filename='mapbox-counties-test.html')

'file:///Users/jackparmer/Documents/mapbox_counties/mapbox-counties-test.html'

In [174]:
years

[1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015]

In [5]:
df_full_data = pd.read_csv('age_adjusted_death_rate.csv')

In [215]:
df_full_data.columns

Index(['Unnamed: 0', '"County"', '"County Code"', '"Year"', '"Year Code"',
       'Deaths', 'Population', 'Crude Rate',
       'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval', 'Age Adjusted Rate',
       'Age Adjusted Rate Lower 95% Confidence Interval',
       'Age Adjusted Rate Upper 95% Confidence Interval'],
      dtype='object')

In [6]:
df_full_data.head()

,Unnamed: 0,"""County""","""County Code""","""Year""","""Year Code""",Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval
0,0,"""Autauga County, AL""",1001,"""1999""","""1999""",NaN,42963,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"""Autauga County, AL""",1001,"""2000""","""2000""",NaN,43671,NaN,NaN,NaN,NaN,NaN,NaN
2,2,"""Autauga County, AL""",1001,"""2001""","""2001""",NaN,44889,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"""Autauga County, AL""",1001,"""2002""","""2002""",NaN,45909,NaN,NaN,NaN,NaN,NaN,NaN
4,4,"""Autauga County, AL""",1001,"""2003""","""2003""",NaN,46800,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
no_quotes = pd.read_csv('age_adjusted_death_rate_no_quotes.csv')

In [4]:
no_quotes.head()

,Unnamed: 0,County,County Code,Year,Year Code,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Age Adjusted Rate,Age Adjusted Rate Lower 95% Confidence Interval,Age Adjusted Rate Upper 95% Confidence Interval
0,Autauga County,AL,1001,1999,1999,NaN,42963,NaN,NaN,NaN,NaN,NaN,NaN
1,Autauga County,AL,1001,2000,2000,NaN,43671,NaN,NaN,NaN,NaN,NaN,NaN
2,Autauga County,AL,1001,2001,2001,NaN,44889,NaN,NaN,NaN,NaN,NaN,NaN
3,Autauga County,AL,1001,2002,2002,NaN,45909,NaN,NaN,NaN,NaN,NaN,NaN
4,Autauga County,AL,1001,2003,2003,NaN,46800,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
no_quotes.columns

Index(['Unnamed: 0', 'County', 'County Code', 'Year', 'Year Code', 'Deaths',
       'Population', 'Crude Rate', 'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval', 'Age Adjusted Rate',
       'Age Adjusted Rate Lower 95% Confidence Interval',
       'Age Adjusted Rate Upper 95% Confidence Interval'],
      dtype='object')

In [9]:
import numpy as np

In [15]:
{str(tick): str(tick) for tick in np.linspace(0,0.1,1)}

{'0.0': '0.0'}

In [18]:
{str(tick)[0:3]: str(tick) for tick in np.linspace(0,1,11)}

{'0.0': '0.0',
 '0.1': '0.1',
 '0.2': '0.2',
 '0.3': '0.30000000000000004',
 '0.4': '0.4',
 '0.5': '0.5',
 '0.6': '0.6000000000000001',
 '0.7': '0.7000000000000001',
 '0.8': '0.8',
 '0.9': '0.9',
 '1.0': '1.0'}

In [22]:
deaths_by_fips = no_quotes.groupby('County Code')['Deaths']

In [35]:
no_quotes['Deaths'] = pd.to_numeric(no_quotes.Deaths, errors='coerce')

In [38]:
deaths_by_fips = no_quotes.groupby('County')['Deaths'].sum()

In [41]:
deaths_by_fips.values.tolist()

[1276.0,
 5927.0,
 2700.0,
 16023.0,
 62895.0,
 10436.0,
 7950.0,
 1622.0,
 2132.0,
 43878.0,
 10448.0,
 2147.0,
 1499.0,
 1604.0,
 20508.0,
 9912.0,
 2680.0,
 8568.0,
 8539.0,
 16720.0,
 14460.0,
 2118.0,
 18497.0,
 4477.0,
 9824.0,
 2158.0,
 832.0,
 14738.0,
 38.0,
 992.0,
 2827.0,
 16183.0,
 6189.0,
 8027.0,
 27530.0,
 26909.0,
 7023.0,
 6279.0,
 33245.0,
 2916.0,
 7448.0,
 217.0,
 12043.0,
 30853.0,
 7018.0,
 7703.0,
 480.0,
 14274.0,
 7936.0,
 5223.0,
 375.0]